# P3 - Data Wrangling (Python, SQL)

## James Cooper

Project Details (Wrangle OpenStreetMap Data):

Location: Glasgow - I chose this city as this is where I live. It's a nice city and I'd like to contribute to the clarity of the data associated with it.

Objective:

Choose any area of the world in https://www.openstreetmap.org and use data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data for a part of the world that you care about.

References:

    Udacity "Data Wrangling with SQL"
    Udacity forums
    Stack Overflow

Contents:

    1 - High Level Overview
    2 - Problems in the data: Audit - Streetnames / Phone Number / Postcode
    3 - Cleaning the data / Importing to SQL / Creating Database
    4 - SQL analysis
    5 - Conclusion


# 1 - High Level Overview

First I will set up the file by importing the required libraries.

In [14]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
from collections import defaultdict
import sqlite3

import cerberus

import schema

We can explore the XML file to get a feel for the contents by counting the top level tags.

In [7]:
#initialise empty dictionary
tags = {}
def count_tags(filename):
        # loop over elementtree object
    for event, elem in ET.iterparse(filename):
        if elem.tag in tags.keys():
            tags[elem.tag]+= 1
        else:
            tags[elem.tag] = 1
    return tags

def count():

    tags = count_tags('G129.osm')
    pprint.pprint(tags)


count()

{'bounds': 1,
 'member': 26514,
 'nd': 657766,
 'node': 577196,
 'osm': 1,
 'relation': 1061,
 'tag': 367900,
 'way': 93605}


The OSM wiki explains the basic structure of the data. In this instance we see that there are just short of 600,000 nodes in the dataset - these represent specific points on the map, and have associated attributes embedded in the node tag itself. Some nodes have children - additional information about each node is embedded in the child tags.

Ways are lists of nodes that make up an area, route or street. Each way also has its own attributes.

Relations are outwith the scope of this analysis.

### Filesizes

In [8]:
import os

print 'OSM filesize = {} MB'.format(os.path.getsize('G129.osm')/1.0e6)
print 'Sample filesize = {} MB'.format(os.path.getsize('G20.osm')/1.0e6)
print 'csv - nodes filesize = {} MB'.format(os.path.getsize('nodes.csv')/1.0e6)
print 'csv - nodes_tags filesize = {} MB'.format(os.path.getsize('nodes_tags.csv')/1.0e6)
print 'csv - ways filesize = {} MB'.format(os.path.getsize('ways.csv')/1.0e6)
print 'csv - ways_nodes filesize = {} MB'.format(os.path.getsize('ways_nodes.csv')/1.0e6)
print 'csv - ways_tags filesize = {} MB'.format(os.path.getsize('ways_tags.csv')/1.0e6)
print 'database filesize = {} MB'.format(os.path.getsize('database.db')/1.0e6)

OSM filesize = 131.786712 MB
Sample filesize = 6.665162 MB
csv - nodes filesize = 47.936789 MB
csv - nodes_tags filesize = 5.368172 MB
csv - ways filesize = 5.620808 MB
csv - ways_nodes filesize = 16.240482 MB
csv - ways_tags filesize = 8.00229 MB
database filesize = 94.425088 MB


### 2Tagtypes.py 

This section is intended to highlight any issues within the k attribute of each tag. The code was provided in the lesson.
Each key is assigned to one of four groups: lower means that the key includes lower case characters.
Lower_colon includes colons. problemchars includes any characters such as [=\+/&<>;\'"\?%#$@\,\. \t\r\n]. other covers any keys not meeting the above criteria.

{'lower': 318604, 'lower_colon': 26284, 'other': 23011, 'problemchars': 1}


We can see that out of the entire dataset only one of the keys includes problematic characters.


### 3Countuser.py

As part of the exercises this funciton was written in order to return the total number of users who had made modifications to in the dataset.

Number of unique users = 818


#  2 - Problems in the data: Audit - Streetnames / Phone Number / Postcode

# Audit.py

Purpose of this function is to return all values not matching a specific format.
The fields I am looking to clean are Streetnames, phone numbers and postcodes.

The code builds on that provided in the exercise  - I have added to the expected list for streetnames to reflect that there are several extra common words used in Glasgow to refer to streets, based on the previous instance of this code which was run on a 5% sample of the full dataset. This will ensure that they are not showing up erroneously in the Audit. 


### Street Types

At first glance, based on the number of returns it looks like the initial expected list was not sufficiently comprehensive - however many of these results are not "Wrong" per se, so much as "reflective of the rich linguistic heritage of scotland". These results do not require cleaning and can be left in the data. However there are also some problems with spelling and consistency (Sreet, Strret etc) - these are errors that can be fixed by adding additional terms to our mapping dictionary in data.py using it to swap in something legible wherever one of these issues is encountered.


### Phone Numbers

The returned numbers fall into several categories.

1. There are various permutations of no spaces, spaces and dashes used at different points in each number.
2. Some numbers include an international dialling code. Others omit this and are in local format.
3. Some have both the international dialing code and the local code fully included within the number 
4. Some phone numbers include brackets or spaces between the international and local codes.
5. Some numbers include multiple separate phone numbers.
6. Some numbers just have the wrong number of numbers.

To ensure consistency I decided to impose the format (+44-nnn-nnn-nnnn). +44 is the international dialing code for the uk and replaces the first 0 in the local number.

To address these points, my function in data.py will take each value and remove all non digits. This removes a lot of the degrees of entropy when dealing with the above issues. From there, we can check the length of the string of the  number and use .startswith in order to match each of the various circumstances identified above. Following this, the number is modified to match my specific format.


### Postcodes

Not much to report here - other than the obvious observation that there a few PA2 postcodes in there. These correspond to addresses in renfrewshire. It was in fact myself who inadvertently caught a section of renfrewshire in the map when selecting a square map area that both met the required filesize and covered the glasgow metropolitan area. These won't impair anyone's ability to use the dataset provided they are aware of how it was selected, so I will leave them in.

With regards to the other results - several results are returning only the high level "Area" section of the postcode, IE only the first 2 or 3 characters. To quote Wikipedia:

"The structure of a postcode is a one or two-letter postcode area code named for a local town or area of London, one or two digits signifying a district in that region, a space, and then an arbitrary code of one number and two letters. For example, the postcode of the University of Roehampton in London is SW15 5PU, where SW stands for south-west London. The postcode of GCHQ is GL51 0EX, where GL signifies the postal town of Gloucester."

In this case as the area of interest is Glasgow, the postcode area code is G. The district code can be one or two digits, and the arbitrary code (after the space or dash) giving the exact delivery address is denoted by Number Letter Letter. So overall some of our returns are only showing the District code - unfortunately there is no way to fix this. I will not remove these however as they still give some indication of where the node is located, which is better than nothing.

Note - I also intend to create Area codes for nodes that already have postcodes. This is because it allows us to search by district, to get a sense of where all the good amenities are located for example.

My code will also modify all existing postcode consisting of 2 or 3 characters to have "---" appended to them so that it is clear that they are incomplete.




# 3 - Cleaning the data / Exporting to CSV / Creating Database

## ShapeElement.py

Problems encountered:

1.

There were issues with missing data causing the reader to crash - this necessitated the use of the following code
 to ensure that the reader did not crash when encountering missing info, and instead returned a placeholder. This is designed to be easily identifiable.
 
```
else:
                #insert placeholder value for missing data
                node_attribs[attribute] = "999999999999"
```

2.

My attempts to create a new element called Areacode based on the first three letters of any postcode were stymied because the values for postcode were being overwritten. This was solved with assistance from the forums and the use of a new dictionary , nodes1, that allowed the new element to be created in the csv for each node with a postcode.

```
elif child.attrib['k'] == "addr:postcode":
                    nodes["value"] = update_postcode(child.attrib['v'])
                    tags.append(nodes)
                    if make_areacode_out_of_postcode(child.attrib['v']):
                        nodes1['type'] = 'addr'
                        nodes1['key'] = 'Areacode'
                        nodes1['id'] = element.attrib['id']
                        nodes1["value"] = make_areacode_out_of_postcode(child.attrib['v']).strip()
                        tags.append(nodes1)                    
```




### Importing CSV to database with required schema

## Create and fill database.py

There were difficulties posed by the format of the csv files - for some reason the parser had decided that there were characters that did not fit the required encoding. This was a surprise as I was unaware of "foreign" characters in the dataset. This was resolved by decoding each column, as shown below in the case of the nodes_tags table:

```
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in dr]

```


# 4 - SQL analysis

## db queries.py

## Top ten cuisines

[(u'chinese', 70), (u'indian', 66), (u'italian', 49), (u'sandwich', 40), (u'fish_and_chips', 31), (u'pizza', 21), (u'burger', 18), (u'coffee_shop', 18), (u'regional', 16), (u'asian', 10)]


Chinese narrowly edges out Indian as glasgow's preferred cuisine.

## Top ten amenities

[(u'bicycle_parking', 556), (u'post_box', 458), (u'fast_food', 307), (u'restaurant', 268), (u'pub', 255), (u'cafe', 229), (u'telephone', 166), (u'atm', 120), (u'bench', 115), (u'recycling', 98)]


Glasgow benefits from a surfeit of bicycle parking.

## Top ten shops

[(u'convenience', 231), (u'hairdresser', 119), (u'clothes', 65), (u'supermarket', 65), (u'bookmaker', 54), (u'newsagent', 50), (u'beauty', 40), (u'bakery', 32), (u'yes', 27), (u'butcher', 24)]


As can be seen, the overwhelming majority of shops in glasgow are convenience stores. There is a surprisingly low number of bookmakers - as this is a proxy for deprivation it would be interesting to explore this further, perhaps by areacode. Outside my flat there are 3 bookkeepers within a 60 second walk.

Note that 'Yes' is getting returned as a type of shop - given that the existence of the shop key implies that a shop exists, this is a potential further opportunity for data cleaning.


## Areacodes with the most Bookmakers

[(u'G1', 3), (u'G2', 1), (u'G4', 1)]


Our query returns only 5 results. This is disappointing given the trouble it took to create the Areacode key.

An examination of the CSV indicates that the vast majority of the bookmakers listed in the data do not have an associated postcode: as a result neither do they have an Areacode. - This isn't a cleaning issue however.

In the interest of getting a meaningful output lets investigate restaurants - perhaps these will have fully completed postcodes.

## Areacodes with the most restaurants


[(u'G1', 13), (u'G2', 7), (u'G3', 6), (u'G12', 3), (u'G31', 3), (u'G4', 2), (u'G41', 2), (u'G53', 2), (u'G20', 1), (u'G33', 1)]


Once again we are challenged by the incompleteness of the data. Despite the previous count showing a total of no less than 268 restaurants, our top ten Areacodes only account for a small proportion of this total. However assuming that the likelihood of the restuarant not having an associated postcode (and thus Areacode) is unaffected by location, we can still conclude that G1 must have a significant proportion of the total.

## Conclusion


As discussed above, there are a few issues about completeness of the data as it relates to the postcodes being completed for each node. This meant that our ability to fully utilise the newly created Areacode key to assist our SQL exploration was impaired.

I noticed during my exploration that the bus stops in the data get their information directly from the NAPTAN (government public transport) database, and potentially using a similar tool -such as the post offices publically available database - could be used to add in these postcode values. There wouldn't be any danger of overriding the wrong values as the post office have the most accurate postcodes and these do not change even if a restaurant changes ownership for example. By automating this, some of the pitfalls of relying on user entry data can be avoided.

This would however also introduce new challenges, such as where to source the information. However some research indicates that there are robust datasets available in a fairly easy to query format - http://www.uk-postcodes.com/api could be used for example. Specifically, there are python libraries available https://postcodes.readthedocs.io/en/latest/ that could be used to find required postcodes by taking the known GPS co-ordinates of any specific node, returning the postcode, and then making a script to append this to the relevant element. This would work reliably because each node has an associated lat and lon value.

Additional improvements could also be made manually within the existing dataset.

Key - Shop value - "yes" - this could be addressed by writing code that detects whether there is a secondary key confirming what type of shop it is. This key could then be removed, and used as the value for the Shop key, ovewrwriting 'yes'.

Phone numbers - certain nodes have multiple phone numbers - This could be addressed by splitting out the second number and reassigning it to another key. I did not code a solution to this as this was not a common issue in the data and the complexity of code required to make this compatible with the treatment of every other phone number made it unrealistic.

